In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import random
import numba
import statsmodels.api as sm
from sklearn.metrics import r2_score
import pandas as pd
import os
import sys

# Step 1: Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "/scratch/ivyglade/pdo/"))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Step 2: Now you can import file.py
import pdo_functions  # assuming file.py contains functions/classes

import importlib
importlib.reload(pdo_functions)

<module 'pdo_functions' from '/scratch/ivyglade/pdo/pdo_functions.py'>

In [2]:
# Open environmental data
cape = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/wrf_cape/era5_daily_max_mucape_*s.nc', combine='nested', concat_dim='year')['__xarray_dataarray_variable__']
s06 = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/era5_s06_*s.nc', combine='nested', concat_dim='time')['__xarray_dataarray_variable__']
ndsev = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ndsev/ndsev_1940-2024_mam_corrected.nc')['__xarray_dataarray_variable__']

In [3]:
# Need to compute monthly means
ndsev_monthly = np.zeros((85, 101, 237))
for i in range(85):
    # Load in one year of data
    ndsev_monthly[i] = ndsev.isel(time=slice(i*92, (i+1)*92)).transpose('time', 'latitude', 'longitude').values.sum(axis=0)
    
    # Progress
    print(f'{1940+i} is complete.')

1940 is complete.
1941 is complete.
1942 is complete.
1943 is complete.
1944 is complete.
1945 is complete.
1946 is complete.
1947 is complete.
1948 is complete.
1949 is complete.
1950 is complete.
1951 is complete.
1952 is complete.
1953 is complete.
1954 is complete.
1955 is complete.
1956 is complete.
1957 is complete.
1958 is complete.
1959 is complete.
1960 is complete.
1961 is complete.
1962 is complete.
1963 is complete.
1964 is complete.
1965 is complete.
1966 is complete.
1967 is complete.
1968 is complete.
1969 is complete.
1970 is complete.
1971 is complete.
1972 is complete.
1973 is complete.
1974 is complete.
1975 is complete.
1976 is complete.
1977 is complete.
1978 is complete.
1979 is complete.
1980 is complete.
1981 is complete.
1982 is complete.
1983 is complete.
1984 is complete.
1985 is complete.
1986 is complete.
1987 is complete.
1988 is complete.
1989 is complete.
1990 is complete.
1991 is complete.
1992 is complete.
1993 is complete.
1994 is complete.
1995 is co

In [4]:
# Select the CONUS - making sure that CAPE, S06 and NDSEV all have the same shape
cape_conus = cape.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
s06_conus  = s06.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))

In [5]:
# Select MAM only
cape_conus_mam = cape_conus.sel(date=cape_conus.date.where(
                                     cape_conus.date.str.startswith('03-') |
                                     cape_conus.date.str.startswith('04-') |
                                     cape_conus.date.str.startswith('05-'),
                                     drop=True)).load()

s06_conus_mam = s06_conus.sel(time=s06_conus.time.dt.month.isin([3, 4, 5]))

In [6]:
# Flatten to have a single time dimension
cape_conus_mam_mean_stack = cape_conus_mam.stack(time=('year', 'date'))

# Convert time to datetime
year = cape_conus_mam_mean_stack['time'].get_index('time').get_level_values('year')
date = cape_conus_mam_mean_stack['time'].get_index('time').get_level_values('date')

datetime = pd.to_datetime(year.astype(str) + '-' + date.astype(str))

cape_conus_mam_mean_stack = cape_conus_mam_mean_stack.assign_coords(time=datetime)

/tmp/ipykernel_27744/906048647.py:10: FutureWarning: updating coordinate 'time' with a PandasMultiIndex would leave the multi-index level coordinates ['year', 'date'] in an inconsistent state. This will raise an error in the future. Use `.drop_vars(['time', 'year', 'date'])` before assigning new coordinate values.
  cape_conus_mam_mean_stack = cape_conus_mam_mean_stack.assign_coords(time=datetime)


In [7]:
# Take MAM mean
s06_conus_mam_mean = s06_conus_mam.resample(time='YE').mean().dropna(dim='time')
cape_conus_mam_mean = cape_conus_mam_mean_stack.resample(time='YE').mean().dropna(dim='time').transpose('time', 'latitude', 'longitude')

In [8]:
# Convert ndsev_monthly to XR
ndsev_conus = xr.DataArray(ndsev_monthly, coords={'time':cape_conus_mam_mean['time'], 'latitude':s06_conus_mam_mean['latitude'], 'longitude':s06_conus_mam_mean['longitude']}, dims=['time', 'latitude', 'longitude'])

In [9]:
# Detrend
cape_conus_mam_de = pdo_functions.detrend_dim(cape_conus_mam_mean, 'time', 2)
s06_conus_mam_de  = pdo_functions.detrend_dim(s06_conus_mam_mean, 'time', 2).load()
ndsev_conus_mam_de = pdo_functions.detrend_dim(ndsev_conus, 'time', 2)

In [10]:
# Open SST data and compute the PDO and Nino3.4
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

pdo = pdo_functions.pdo_from_hadisst(sst, 1870, 2024)

oni = pdo_functions.oni_moving_base(sst)

In [11]:
# 3- month rolling mean
oni_rolling = oni.rolling(time=3, center=True).mean()
oni_rolling_1940_2024 = oni_rolling.sel(time=oni_rolling.time.dt.year.isin(np.arange(1940, 2025, 1)))
oni_rolling_1940_2024_mam = oni_rolling_1940_2024.sel(time=oni_rolling_1940_2024.time.dt.month.isin([3, 4, 5])).resample(time='YE').mean()

# Calculating when El Nino and La Nina events occur
oni_calc = np.zeros((1668))
for i in range(1668):
    if i < 5:
        continue
    else:
        if (oni_rolling[(i-4)] > 0.5) & (oni_rolling[(i-3)] > 0.5) & (oni_rolling[(i-2)] > 0.5) & (oni_rolling[(i-1)] > 0.5) & (oni_rolling[i] > 0.5):
            oni_calc[i] = 1
        elif (oni_rolling[(i-4)] < -0.5) & (oni_rolling[(i-3)] < -0.5) & (oni_rolling[(i-2)] < -0.5) & (oni_rolling[(i-1)] < -0.5) & (oni_rolling[i] < -0.5):
            oni_calc[i] = -1
        else:
            continue

# Coonvert to XR
oni_calc_xr = xr.DataArray(oni_calc, coords={'time':oni['time']}, dims=['time'])

# 1940-2024 only
oni_1940_2024 = oni_calc_xr.sel(time=oni_calc_xr.time.dt.year.isin(np.arange(1940, 2025, 1)))
oni_1940_2024_mam = oni_1940_2024.sel(time=oni_1940_2024.time.dt.month.isin([3, 4, 5]))

# Need to calculate when El Nino/La Nina Occurs in a given year
oni_1940_2024_mam_ann = np.zeros((85))
for i in range(85):
    if oni_1940_2024_mam[i*3: (i+1)*3].sum(axis=0) > 0:
        oni_1940_2024_mam_ann[i] = 1

    elif oni_1940_2024_mam[i*3: (i+1)*3].sum(axis=0) < 0:
        oni_1940_2024_mam_ann[i] = -1

    else:
        continue

In [12]:
# Convert pdo to xarray
pdo_xr = xr.DataArray(pdo, coords={'time':sst['time']}, dims=['time'])

# Subset only 1940-2024
pdo_1940_2024 = pdo_xr.sel(time=pdo_xr.time.dt.year.isin(np.arange(1940, 2025, 1)))

# only MAM and take seasonal averages
pdo_1940_2024_mam = pdo_1940_2024.sel(time=pdo_1940_2024.time.dt.month.isin([3, 4, 5])).resample(time='YE').mean()

# aligning time arrays with that of CAPE
pdo_1940_2024_mam['time'] = cape_conus_mam_de['time']

s06_conus_mam_de['time'] = cape_conus_mam_de['time']
oni_rolling_1940_2024_mam['time'] = cape_conus_mam_de['time']

oni_ann_xr = xr.DataArray(oni_1940_2024_mam_ann, coords={'time':cape_conus_mam_de['time']}, dims=['time'])

In [13]:
# Take the 5 yr rolling mean of each convective parameter
cape_conus_mam_rolling = cape_conus_mam_de.rolling(time=5, center=True).mean()
s06_conus_mam_rolling = s06_conus_mam_de.rolling(time=5, center=True).mean()
ndsev_conus_mam_rolling = ndsev_conus_mam_de.rolling(time=5, center=True).mean()

In [26]:
# Compositing
cape_pos_pdo = xr.where(pdo_1940_2024_mam > 0.5, cape_conus_mam_de, np.nan).dropna(dim='time')
cape_neg_pdo = xr.where(pdo_1940_2024_mam < -1, cape_conus_mam_de, np.nan).dropna(dim='time')

cape_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, cape_conus_mam_de, np.nan).dropna(dim='time')
cape_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, cape_conus_mam_de, np.nan).dropna(dim='time')

cape_pos_nino = xr.where(oni_ann_xr == 1, cape_conus_mam_de, np.nan).dropna(dim='time')
cape_neg_nina = xr.where(oni_ann_xr == -1, cape_conus_mam_de, np.nan).dropna(dim='time')

cape_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), cape_conus_mam_de, np.nan).dropna(dim='time')
cape_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), cape_conus_mam_de, np.nan).dropna(dim='time')

s06_pos_pdo = xr.where(pdo_1940_2024_mam > 0, s06_conus_mam_de, np.nan).dropna(dim='time')
s06_neg_pdo = xr.where(pdo_1940_2024_mam < 0, s06_conus_mam_de, np.nan).dropna(dim='time')

s06_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, s06_conus_mam_de, np.nan).dropna(dim='time')
s06_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, s06_conus_mam_de, np.nan).dropna(dim='time')

s06_pos_nino = xr.where(oni_ann_xr == 1, s06_conus_mam_de, np.nan).dropna(dim='time')
s06_neg_nina = xr.where(oni_ann_xr == -1, s06_conus_mam_de, np.nan).dropna(dim='time')

s06_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), s06_conus_mam_de, np.nan).dropna(dim='time')
s06_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), s06_conus_mam_de, np.nan).dropna(dim='time')

ndsev_pos_pdo = xr.where(pdo_1940_2024_mam > 0, ndsev_conus_mam_de, np.nan).dropna(dim='time')
ndsev_neg_pdo = xr.where(pdo_1940_2024_mam < 0, ndsev_conus_mam_de, np.nan).dropna(dim='time')

ndsev_pos_oni = xr.where(oni_rolling_1940_2024_mam > 0, ndsev_conus_mam_de, np.nan).dropna(dim='time')
ndsev_neg_oni = xr.where(oni_rolling_1940_2024_mam < 0, ndsev_conus_mam_de, np.nan).dropna(dim='time')

ndsev_pos_nino = xr.where(oni_ann_xr == 1, ndsev_conus_mam_de, np.nan).dropna(dim='time')
ndsev_neg_nina = xr.where(oni_ann_xr == -1, ndsev_conus_mam_de, np.nan).dropna(dim='time')

ndsev_pos_pdo_neut_oni = xr.where((pdo_1940_2024_mam > 0) & (oni_ann_xr == 0), ndsev_conus_mam_de, np.nan).dropna(dim='time')
ndsev_neg_pdo_neut_oni = xr.where((pdo_1940_2024_mam < 0) & (oni_ann_xr == 0), ndsev_conus_mam_de, np.nan).dropna(dim='time')

In [28]:
pdo_1940_2024_mam

<xarray.DataArray 'pcs' (time: 85)> Size: 680B
array([ 2.06979154,  2.30041754,  1.31274499,  0.36866846, -0.19108185,
       -0.70687865, -0.43917372,  0.63116982, -0.86587283, -1.47777883,
       -1.97000336, -1.10133973, -1.11092489, -0.470946  , -0.56853584,
       -1.50912795, -2.12763699, -0.36164671,  0.97628665,  0.02652579,
        0.56180074,  0.54971538, -0.92776106, -0.43680926, -1.30688533,
       -0.35182484, -0.42664903, -1.13858581, -0.5505415 , -0.68534237,
        0.43419494, -1.40602491, -1.48138297, -0.68195631, -0.78740648,
       -1.32677577, -1.03563773,  0.38181564,  1.19840773,  0.25978107,
        1.11407507,  1.62969246, -0.38506649,  1.72966315,  1.18413669,
       -0.12301461,  1.31095123,  1.87986766,  0.92153057, -0.42242508,
       -0.16835406, -1.08624588,  0.97951051,  1.16734285,  0.69177362,
        0.70170274,  1.40054309,  0.94724474,  1.26047973, -0.74578751,
       -0.14924458, -0.05432139, -0.38919377,  0.7632802 ,  0.50507554,
        0.81379006,  0.07670946, -0.20990401, -1.25266247, -1.16222663,
        0.32000808, -0.8441555 , -0.91124693, -0.36816103,  0.86182328,
        0.96356818,  2.09101329,  0.53529601, -0.32963375,  0.41972092,
       -0.94980483, -1.27396353, -1.62006674, -1.53809892, -1.03139989])
Coordinates:
  * time     (time) datetime64[ns] 680B 1940-12-31 1941-12-31 ... 2024-12-31

In [ ]:
# Compositing - rolling mean
cape_pos_pdo_rolling = xr.where(pdo_1940_2024_mam[2:-2] > 0, cape_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
cape_neg_pdo_rolling = xr.where(pdo_1940_2024_mam[2:-2] < 0, cape_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

cape_pos_oni_rolling = xr.where(oni_rolling_1940_2024_mam[2:-2] > 0, cape_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
cape_neg_oni_rolling = xr.where(oni_rolling_1940_2024_mam[2:-2] < 0, cape_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

cape_pos_nino_rolling = xr.where(oni_ann_xr[2:-2] == 1, cape_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
cape_neg_nina_rolling = xr.where(oni_ann_xr[2:-2] == -1, cape_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

cape_pos_pdo_neut_oni_rolling = xr.where((pdo_1940_2024_mam[2:-2] > 0) & (oni_ann_xr[2:-2] == 0), cape_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
cape_neg_pdo_neut_oni_rolling = xr.where((pdo_1940_2024_mam[2:-2] < 0) & (oni_ann_xr[2:-2] == 0), cape_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

s06_pos_pdo_rolling = xr.where(pdo_1940_2024_mam[2:-2] > 0, s06_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
s06_neg_pdo_rolling = xr.where(pdo_1940_2024_mam[2:-2] < 0, s06_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

s06_pos_oni_rolling = xr.where(oni_rolling_1940_2024_mam[2:-2] > 0, s06_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
s06_neg_oni_rolling = xr.where(oni_rolling_1940_2024_mam[2:-2] < 0, s06_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

s06_pos_nino_rolling = xr.where(oni_ann_xr[2:-2] == 1, s06_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
s06_neg_nina_rolling = xr.where(oni_ann_xr[2:-2] == -1, s06_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

s06_pos_pdo_neut_oni_rolling = xr.where((pdo_1940_2024_mam[2:-2] > 0) & (oni_ann_xr[2:-2] == 0), s06_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
s06_neg_pdo_neut_oni_rolling = xr.where((pdo_1940_2024_mam[2:-2] < 0) & (oni_ann_xr[2:-2] == 0), s06_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

ndsev_pos_pdo_rolling = xr.where(pdo_1940_2024_mam[2:-2] > 0, ndsev_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
ndsev_neg_pdo_rolling = xr.where(pdo_1940_2024_mam[2:-2] < 0, ndsev_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

ndsev_pos_oni_rolling = xr.where(oni_rolling_1940_2024_mam[2:-2] > 0, ndsev_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
ndsev_neg_oni_rolling = xr.where(oni_rolling_1940_2024_mam[2:-2] < 0, ndsev_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

ndsev_pos_nino_rolling = xr.where(oni_ann_xr[2:-2] == 1, ndsev_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
ndsev_neg_nina_rolling = xr.where(oni_ann_xr[2:-2] == -1, ndsev_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

ndsev_pos_pdo_neut_oni_rolling = xr.where((pdo_1940_2024_mam[2:-2] > 0) & (oni_ann_xr[2:-2] == 0), ndsev_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')
ndsev_neg_pdo_neut_oni_rolling = xr.where((pdo_1940_2024_mam[2:-2] < 0) & (oni_ann_xr[2:-2] == 0), ndsev_conus_mam_rolling[2:-2], np.nan).dropna(dim='time')

In [15]:
def bootstrap_p_values_fast(sample_mean, cape_climo, comp_cond, n_boot=10_000):
    n_lats, n_lons, n_time = cape_climo.shape
    n_comp = np.count_nonzero(comp_cond)

    # Flatten the spatial dimensions
    grid_size = n_lats * n_lons
    cape_flat = cape_climo.reshape(grid_size, n_time)
    sample_mean_flat = sample_mean.ravel()

    # Draw bootstrap samples for all grid points at once
    rand_idx = np.random.randint(0, n_time, size=(n_boot, n_comp))
    boot_samples = cape_flat[:, rand_idx.T]  # shape: (grid_size, n_comp, n_boot)
    boot_samples = boot_samples.transpose(0, 2, 1)  # shape: (grid_size, n_boot, n_comp)

    # Mean across composite months
    boot_means = boot_samples.mean(axis=2)  # shape: (grid_size, n_boot)

    # Climatology mean for each grid point
    climo_mean = cape_flat.mean(axis=1)  # shape: (grid_size,)

    # Calculate p-values
    diffs = np.abs(boot_means - climo_mean[:, None])  # shape: (grid_size, n_boot)
    test_diffs = np.abs(sample_mean_flat - climo_mean)
    p_values_flat = np.mean(diffs >= test_diffs[:, None], axis=1)

    # Confidence intervals
    lower_flat = np.percentile(boot_means, 2.5, axis=1)
    upper_flat = np.percentile(boot_means, 97.5, axis=1)

    # Reshape back to (lat, lon)
    p_values = p_values_flat.reshape(n_lats, n_lons)
    lower_grid = lower_flat.reshape(n_lats, n_lons)
    upper_grid = upper_flat.reshape(n_lats, n_lons)

    return p_values, upper_grid, lower_grid

In [16]:
cape_pos_pdo_p_fast, cape_pos_pdo_upper_fast, cape_pos_pdo_lower_fast = bootstrap_p_values_fast(cape_pos_pdo.values.mean(axis=0), cape_conus_mam_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values > 0)

In [23]:
np.shape(cape_pos_pdo)

(14, 101, 237)

In [ ]:
cape_pos_pdo_neut_nino_p_fast, cape_pos_pdo_neut_nino_upper_fast, cape_pos_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(cape_pos_pdo_neut_oni.values.mean(axis=0), cape_conus_mam_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam > 0) & (oni_ann_xr == 0))

In [ ]:
cape_neg_pdo_p_fast, cape_neg_pdo_upper_fast, cape_neg_pdo_lower_fast = bootstrap_p_values_fast(cape_neg_pdo.values.mean(axis=0), cape_conus_mam_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values < 0)

In [ ]:
cape_neg_pdo_neut_nino_p_fast, cape_neg_pdo_neut_nino_upper_fast, cape_neg_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(cape_neg_pdo_neut_oni.values.mean(axis=0), cape_conus_mam_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam < 0) & (oni_ann_xr == 0))

In [ ]:
# cape_pos_nino_p_fast, cape_pos_nino_upper_fast, cape_pos_nino_lower_fast = bootstrap_p_values_fast(cape_pos_nino.values.mean(axis=0), cape_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == 1)

In [ ]:
# cape_neg_nino_p_fast, cape_neg_nino_upper_fast, cape_neg_nino_lower_fast = bootstrap_p_values_fast(cape_neg_nina.values.mean(axis=0), cape_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == -1)

In [ ]:
# cape_pos_oni_p_fast, cape_pos_oni_upper_fast, cape_pos_oni_lower_fast = bootstrap_p_values_fast(cape_pos_oni.values.mean(axis=0), cape_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam > 0)

In [ ]:
# cape_neg_oni_p_fast, cape_neg_oni_upper_fast, cape_neg_oni_lower_fast = bootstrap_p_values_fast(cape_neg_oni.values.mean(axis=0), cape_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam < 0)

In [ ]:
## Spacer ##

In [ ]:
s06_pos_pdo_p_fast, s06_pos_pdo_upper_fast, s06_pos_pdo_lower_fast = bootstrap_p_values_fast(s06_pos_pdo.values.mean(axis=0), s06_conus_mam_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values > 0)

In [ ]:
s06_pos_pdo_neut_nino_p_fast, s06_pos_pdo_neut_nino_upper_fast, s06_pos_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(s06_pos_pdo_neut_oni.values.mean(axis=0), s06_conus_mam_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam > 0) & (oni_ann_xr == 0))

In [ ]:
s06_neg_pdo_p_fast, s06_neg_pdo_upper_fast, s06_neg_pdo_lower_fast = bootstrap_p_values_fast(s06_neg_pdo.values.mean(axis=0), s06_conus_mam_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values < 0)

In [ ]:
s06_neg_pdo_neut_nino_p_fast, s06_neg_pdo_neut_nino_upper_fast, s06_neg_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(s06_neg_pdo_neut_oni.values.mean(axis=0), s06_conus_mam_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam < 0) & (oni_ann_xr == 0))

In [ ]:
# s06_pos_nino_p_fast, s06_pos_nino_upper_fast, s06_pos_nino_lower_fast = bootstrap_p_values_fast(s06_pos_nino.values.mean(axis=0), s06_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == 1)

In [ ]:
# s06_neg_nino_p_fast, s06_neg_nino_upper_fast, s06_neg_nino_lower_fast = bootstrap_p_values_fast(s06_neg_nina.values.mean(axis=0), s06_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == -1)

In [ ]:
# s06_pos_oni_p_fast, s06_pos_oni_upper_fast, s06_pos_oni_lower_fast = bootstrap_p_values_fast(s06_pos_oni.values.mean(axis=0), s06_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam > 0)

In [ ]:
# s06_neg_oni_p_fast, s06_neg_oni_upper_fast, s06_neg_oni_lower_fast = bootstrap_p_values_fast(s06_neg_oni.values.mean(axis=0), s06_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam < 0)

In [ ]:
## Spacer ##

In [ ]:
ndsev_pos_pdo_p_fast, ndsev_pos_pdo_upper_fast, ndsev_pos_pdo_lower_fast = bootstrap_p_values_fast(ndsev_pos_pdo.values.mean(axis=0), ndsev_conus_mam_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values > 0)

In [ ]:
ndsev_pos_pdo_neut_nino_p_fast, ndsev_pos_pdo_neut_nino_upper_fast, ndsev_pos_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(ndsev_pos_pdo_neut_oni.values.mean(axis=0), ndsev_conus_mam_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam > 0) & (oni_ann_xr == 0))

In [ ]:
ndsev_neg_pdo_p_fast, ndsev_neg_pdo_upper_fast, ndsev_neg_pdo_lower_fast = bootstrap_p_values_fast(ndsev_neg_pdo.values.mean(axis=0), ndsev_conus_mam_de.transpose('latitude', 'longitude', 'time').values, pdo_1940_2024_mam.values < 0)

In [ ]:
ndsev_neg_pdo_neut_nino_p_fast, ndsev_neg_pdo_neut_nino_upper_fast, ndsev_neg_pdo_neut_nino_lower_fast = bootstrap_p_values_fast(ndsev_neg_pdo_neut_oni.values.mean(axis=0), ndsev_conus_mam_de.transpose('latitude', 'longitude', 'time').values, (pdo_1940_2024_mam < 0) & (oni_ann_xr == 0))

In [ ]:
# ndsev_pos_nino_p_fast, ndsev_pos_nino_upper_fast, ndsev_pos_nino_lower_fast = bootstrap_p_values_fast(ndsev_pos_nino.values.mean(axis=0), ndsev_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == 1)

In [ ]:
# ndsev_neg_nino_p_fast, ndsev_neg_nino_upper_fast, ndsev_neg_nino_lower_fast = bootstrap_p_values_fast(ndsev_neg_nina.values.mean(axis=0), ndsev_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_ann_xr.values == -1)

In [ ]:
# ndsev_pos_oni_p_fast, ndsev_pos_oni_upper_fast, ndsev_pos_oni_lower_fast = bootstrap_p_values_fast(ndsev_pos_oni.values.mean(axis=0), ndsev_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam > 0)

In [ ]:
# ndsev_neg_oni_p_fast, ndsev_neg_oni_upper_fast, ndsev_neg_oni_lower_fast = bootstrap_p_values_fast(ndsev_neg_oni.values.mean(axis=0), ndsev_conus_mam_de.transpose('latitude', 'longitude', 'time').values, oni_rolling_1940_2024_mam < 0)

In [ ]:
## Spacer ##

In [21]:
cape_pos_pdo_adj_p = pdo_functions.control_FDR(cape_pos_pdo_p_fast, 101, 237, 0.05)
cape_neg_pdo_adj_p = pdo_functions.control_FDR(cape_neg_pdo_p_fast, 101, 237, 0.2)
cape_pos_pdo_neut_nino_adj_p = pdo_functions.control_FDR(cape_pos_pdo_neut_nino_p_fast, 101, 237, 0.2)
cape_neg_pdo_neut_nino_adj_p = pdo_functions.control_FDR(cape_neg_pdo_neut_nino_p_fast, 101, 237, 0.2)
# cape_pos_oni_adj_p = pdo_functions.control_FDR(cape_pos_oni_p_fast, 101, 237, 0.1)
# cape_neg_oni_adj_p = pdo_functions.control_FDR(cape_neg_oni_p_fast, 101, 237, 0.1)
# cape_pos_nino_adj_p = pdo_functions.control_FDR(cape_pos_nino_p_fast, 101, 237, 0.1)
# cape_neg_nino_adj_p = pdo_functions.control_FDR(cape_neg_nino_p_fast, 101, 237, 0.1)

s06_pos_pdo_adj_p = pdo_functions.control_FDR(s06_pos_pdo_p_fast, 101, 237, 0.2)
s06_neg_pdo_adj_p = pdo_functions.control_FDR(s06_neg_pdo_p_fast, 101, 237, 0.2)
s06_pos_pdo_neut_nino_adj_p = pdo_functions.control_FDR(s06_pos_pdo_neut_nino_p_fast, 101, 237, 0.2)
s06_neg_pdo_neut_nino_adj_p = pdo_functions.control_FDR(s06_neg_pdo_neut_nino_p_fast, 101, 237, 0.2)
# s06_pos_oni_adj_p = pdo_functions.control_FDR(s06_pos_oni_p_fast, 101, 237, 0.1)
# s06_neg_oni_adj_p = pdo_functions.control_FDR(s06_neg_oni_p_fast, 101, 237, 0.1)
# s06_pos_nino_adj_p = pdo_functions.control_FDR(s06_pos_nino_p_fast, 101, 237, 0.1)
# s06_neg_nino_adj_p = pdo_functions.control_FDR(s06_neg_nino_p_fast, 101, 237, 0.1)

ndsev_pos_pdo_adj_p = pdo_functions.control_FDR(ndsev_pos_pdo_p_fast, 101, 237, 0.15)
ndsev_neg_pdo_adj_p = pdo_functions.control_FDR(ndsev_neg_pdo_p_fast, 101, 237, 0.15)
ndsev_pos_pdo_neut_nino_adj_p = pdo_functions.control_FDR(ndsev_pos_pdo_neut_nino_p_fast, 101, 237, 0.2)
ndsev_neg_pdo_neut_nino_adj_p = pdo_functions.control_FDR(ndsev_neg_pdo_neut_nino_p_fast, 101, 237, 0.2)
# ndsev_pos_oni_adj_p = pdo_functions.control_FDR(ndsev_pos_oni_p_fast, 101, 237, 0.1)
# ndsev_neg_oni_adj_p = pdo_functions.control_FDR(ndsev_neg_oni_p_fast, 101, 237, 0.1)
# ndsev_pos_nino_adj_p = pdo_functions.control_FDR(ndsev_pos_nino_p_fast, 101, 237, 0.1)
# ndsev_neg_nino_adj_p = pdo_functions.control_FDR(ndsev_neg_nino_p_fast, 101, 237, 0.1)

NameError: name 'cape_neg_pdo_p_fast' is not defined

In [22]:
cape_pos_pdo_adj_p

0.0201

In [ ]:
cape_neg_pdo_adj_p

In [ ]:
s06_pos_pdo_adj_p

In [ ]:
s06_neg_pdo_adj_p

In [ ]:
ndsev_pos_pdo_adj_p

In [ ]:
ndsev_neg_pdo_adj_p

In [ ]:
## Spacer ##

In [ ]:
# cape_pos_pdo_sig = np.where(cape_pos_pdo_p_fast < cape_pos_pdo_adj_p, 1, 0)
# cape_neg_pdo_sig = np.where(cape_neg_pdo_p_fast < cape_neg_pdo_adj_p, 1, 0)
# cape_pos_pdo_neut_nino_sig = np.where(cape_pos_pdo_neut_nino_p_fast < cape_pos_pdo_neut_nino_adj_p, 1, 0)
# cape_neg_pdo_neut_nino_sig = np.where(cape_neg_pdo_neut_nino_p_fast < cape_neg_pdo_neut_nino_adj_p, 1, 0)
# cape_pos_oni_sig = np.where(cape_pos_oni_p_fast < cape_pos_oni_adj_p, 1, 0)
# cape_neg_oni_sig = np.where(cape_neg_oni_p_fast < cape_neg_oni_adj_p, 1, 0)
# cape_pos_nino_sig = np.where(cape_pos_nino_p_fast < cape_pos_nino_adj_p, 1, 0)
# cape_neg_nina_sig = np.where(cape_neg_nino_p_fast < cape_neg_nino_adj_p, 1, 0)

# s06_pos_pdo_sig = np.where(s06_pos_pdo_p_fast < s06_pos_pdo_adj_p, 1, 0)
# s06_neg_pdo_sig = np.where(s06_neg_pdo_p_fast < s06_neg_pdo_adj_p, 1, 0)
# s06_pos_pdo_neut_nino_sig = np.where(s06_pos_pdo_neut_nino_p_fast < s06_pos_pdo_neut_nino_adj_p, 1, 0)
# s06_neg_pdo_neut_nino_sig = np.where(s06_neg_pdo_neut_nino_p_fast < s06_neg_pdo_neut_nino_adj_p, 1, 0)
# s06_pos_oni_sig = np.where(s06_pos_oni_p_fast < s06_pos_oni_adj_p, 1, 0)
# s06_neg_oni_sig = np.where(s06_neg_oni_p_fast < s06_neg_oni_adj_p, 1, 0)
# s06_pos_nino_sig = np.where(s06_pos_nino_p_fast < s06_pos_nino_adj_p, 1, 0)
# s06_neg_nina_sig = np.where(s06_neg_nino_p_fast < s06_neg_nino_adj_p, 1, 0)

# ndsev_pos_pdo_sig = np.where(ndsev_pos_pdo_p_fast < ndsev_pos_pdo_adj_p, 1, 0)
# ndsev_neg_pdo_sig = np.where(ndsev_neg_pdo_p_fast < ndsev_neg_pdo_adj_p, 1, 0)
# ndsev_pos_pdo_neut_nino_sig = np.where(ndsev_pos_pdo_neut_nino_p_fast < ndsev_pos_pdo_neut_nino_adj_p, 1, 0)
# ndsev_neg_pdo_neut_nino_sig = np.where(ndsev_neg_pdo_neut_nino_p_fast < ndsev_neg_pdo_neut_nino_adj_p, 1, 0)
# ndsev_pos_oni_sig = np.where(ndsev_pos_oni_p_fast < ndsev_pos_oni_adj_p, 1, 0)
# ndsev_neg_oni_sig = np.where(ndsev_neg_oni_p_fast < ndsev_neg_oni_adj_p, 1, 0)
# ndsev_pos_nino_sig = np.where(ndsev_pos_nino_p_fast < ndsev_pos_nino_adj_p, 1, 0)
# ndsev_neg_nina_sig = np.where(ndsev_neg_nino_p_fast < ndsev_neg_nino_adj_p, 1, 0)

In [ ]:
# Land mask
land_110 = regionmask.defined_regions.natural_earth_v4_1_0.land_110
land = xr.where(land_110.mask_3D(cape_conus_mam_de, lon_name='longitude', lat_name='latitude')==True, 1, np.nan).squeeze()

In [ ]:
fig, ax = plt.subplots(2, 3, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ndsev_bounds = [-2, -1.8, -1.6, -1.4, -1.2, -1, -0.8, -0.6, -0.4, -0.2, -0.05, 0.05, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]
ndsev_norm = c.BoundaryNorm(ndsev_bounds, plt.get_cmap('cmr.fusion_r').N)

cape_bounds = [-50, -45, -40, -35, -30, -25, -20, -15, -10, -5, -1, 1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
# cape_bounds = [-100, -90, -80, -70, -60, -50, -40, -30, -20, -10, -2, 2, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
cape_norm = c.BoundaryNorm(cape_bounds, plt.get_cmap('cmr.fusion_r').N)

s06_bounds = [-1.5, -1.35, -1.2, -1.05, -0.9, -0.75, -0.6, -0.45, -0.3, -0.15, -0.03, 0.03, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05, 1.2, 1.35, 1.5]
s06_norm = c.BoundaryNorm(s06_bounds, plt.get_cmap('cmr.fusion_r').N)

ax = [ax[0, 0], ax[0, 1], ax[0, 2], \
      ax[1, 0], ax[1, 1], ax[1, 2]]

lon = ndsev['longitude']
lat = ndsev['latitude']

letters = ['a', 'b', 'c', \
           'd', 'e', 'f']

for i in range(6):
    ax[i].coastlines(lw=0.25, color='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0)
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].set_facecolor('xkcd:light gray')
    ax[i].text(-2700000, 1420000, letters[i], fontweight='normal', size=10)

ax[2].pcolormesh(lon, lat, ndsev_pos_pdo_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_norm)
# ax[5].pcolormesh(lon, lat, ndsev_pos_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_norm)
ax[5].pcolormesh(lon, lat, ndsev_neg_pdo_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_norm)
# ax[11].pcolormesh(lon, lat, ndsev_neg_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_norm)

ax[0].pcolormesh(lon, lat, cape_pos_pdo_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_norm)
# ax[3].pcolormesh(lon, lat, cape_pos_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_norm)
ax[3].pcolormesh(lon, lat, cape_neg_pdo_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_norm)
# ax[9].pcolormesh(lon, lat, cape_neg_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_norm)

ax[1].pcolormesh(lon, lat, s06_pos_pdo_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_norm)
# ax[4].pcolormesh(lon, lat, s06_pos_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_norm)
ax[4].pcolormesh(lon, lat, s06_neg_pdo_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_norm)
# ax[10].pcolormesh(lon, lat, s06_neg_pdo_neut_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_norm)

ax[2].set_title('NDSEV', pad=3, fontsize=10)
ax[0].set_title('CAPE', pad=3, fontsize=10)
ax[1].set_title('S06', pad=3, fontsize=10)

ax[0].text(-3700000, -100000, f"(+) PDO\nn={len(cape_pos_pdo_rolling)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
# ax[3].text(-4000000, -400000, f"(+) PDO &\nneutral ONI\nn={len(cape_pos_pdo_neut_oni)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[3].text(-3700000, -100000, f"(-) PDO\nn={len(cape_neg_pdo_rolling)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
# ax[9].text(-4000000, -400000, f"(-) PDO &\nneutral ONI\nn={len(cape_neg_pdo_neut_oni)}", fontweight='normal', size=10, rotation='horizontal', ha='center')


cax = plt.axes([0.02, .23, 0.29, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=cape_norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', \
                   ticks=[-100, -50, 0, 50, 100])
cbar.set_label(r'J kg$^{-1}$', size=10, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.25)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

cax2 = plt.axes([0.355, .23, 0.29, 0.02])
cbar2 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=s06_norm), cax=cax2, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-1.5, -0.75, 0, 0.75, 1.5])
cbar2.set_label(r'm s$^{-1}$', size=10, fontweight='normal', color='black')
cbar2.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar2.outline.set_linewidth(0.25)
cbar2.outline.set_color('black')
# cbar.outline.set_visible(False)

cax3 = plt.axes([0.695, .23, 0.29, 0.02])
cbar3 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=ndsev_norm), cax=cax3, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-2, -1, 0, 1, 2])
cbar3.set_label(r'days MAM$^{-1}$', size=10, fontweight='normal', color='black')
cbar3.ax.tick_params(which='both', labelsize=10, width=0.25, length=0, labelcolor='black')
cbar3.outline.set_linewidth(0.25)
cbar3.outline.set_color('black')
# cbar.outline.set_visible(False)

lat_min, lat_max = 30, 47
lon_min, lon_max = pdo_functions.convert_longitudes(-105), pdo_functions.convert_longitudes(-82)
rect_lons = [lon_min, lon_max, lon_max, lon_min, lon_min]
rect_lats = [lat_min, lat_min, lat_max, lat_max, lat_min]
ax[2].plot(rect_lons, rect_lats, color='black', lw=1, transform=ccrs.PlateCarree())

mpl.rcParams['hatch.color'] = 'xkcd:gunmetal'
mpl.rcParams['hatch.linewidth'] = 0.375

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=0.75, 
                    wspace=0.05, 
                    hspace=0.05)

In [ ]:
fig, ax = plt.subplots(2, 3, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ndsev_bounds = [-2, -1.8, -1.6, -1.4, -1.2, -1, -0.8, -0.6, -0.4, -0.2, -0.05, 0.05, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]
ndsev_norm = c.BoundaryNorm(ndsev_bounds, plt.get_cmap('cmr.fusion_r').N)

cape_bounds = [-100, -90, -80, -70, -60, -50, -40, -30, -20, -10, -2, 2, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
cape_norm = c.BoundaryNorm(cape_bounds, plt.get_cmap('cmr.fusion_r').N)

s06_bounds = [-1.5, -1.35, -1.2, -1.05, -0.9, -0.75, -0.6, -0.45, -0.3, -0.15, -0.03, 0.03, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05, 1.2, 1.35, 1.5]
s06_norm = c.BoundaryNorm(s06_bounds, plt.get_cmap('cmr.fusion_r').N)

ax = [ax[0, 0], ax[0, 1], ax[0, 2], \
      ax[1, 0], ax[1, 1], ax[1, 2]]

lon = ndsev['longitude']
lat = ndsev['latitude']

letters = ['a', 'b', 'c', \
           'd', 'e', 'f']

for i in range(6):
    ax[i].coastlines(lw=0.25, color='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0)
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].set_facecolor('xkcd:light gray')
    ax[i].text(-2700000, 1420000, letters[i], fontweight='normal', size=10)

ax[2].pcolormesh(lon, lat, ndsev_pos_pdo_neut_oni_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_norm)
ax[5].pcolormesh(lon, lat, ndsev_neg_pdo_neut_oni_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_norm)

ax[0].pcolormesh(lon, lat, cape_pos_pdo_neut_oni_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_norm)
ax[3].pcolormesh(lon, lat, cape_neg_pdo_neut_oni_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_norm)

ax[1].pcolormesh(lon, lat, s06_pos_pdo_neut_oni_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_norm)
ax[4].pcolormesh(lon, lat, s06_neg_pdo_neut_oni_rolling.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_norm)

ax[2].set_title('NDSEV', pad=3, fontsize=10)
ax[0].set_title('CAPE', pad=3, fontsize=10)
ax[1].set_title('S06', pad=3, fontsize=10)

ax[0].text(-4000000, -400000, f"(+) PDO &\nneutral ONI\nn={len(cape_pos_pdo_neut_oni_rolling)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[3].text(-4000000, -400000, f"(-) PDO &\nneutral ONI\nn={len(cape_neg_pdo_neut_oni_rolling)}", fontweight='normal', size=10, rotation='horizontal', ha='center')


cax = plt.axes([0.02, .23, 0.29, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=cape_norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', \
                   ticks=[-100, -50, 0, 50, 100])
cbar.set_label(r'J kg$^{-1}$', size=10, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.25)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

cax2 = plt.axes([0.355, .23, 0.29, 0.02])
cbar2 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=s06_norm), cax=cax2, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-1.5, -0.75, 0, 0.75, 1.5])
cbar2.set_label(r'm s$^{-1}$', size=10, fontweight='normal', color='black')
cbar2.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar2.outline.set_linewidth(0.25)
cbar2.outline.set_color('black')
# cbar.outline.set_visible(False)

cax3 = plt.axes([0.695, .23, 0.29, 0.02])
cbar3 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=ndsev_norm), cax=cax3, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-2, -1, 0, 1, 2])
cbar3.set_label(r'days MAM$^{-1}$', size=10, fontweight='normal', color='black')
cbar3.ax.tick_params(which='both', labelsize=10, width=0.25, length=0, labelcolor='black')
cbar3.outline.set_linewidth(0.25)
cbar3.outline.set_color('black')
# cbar.outline.set_visible(False)

# lat_min, lat_max = 30, 47
# lon_min, lon_max = pdo_functions.convert_longitudes(-105), pdo_functions.convert_longitudes(-82)
# rect_lons = [lon_min, lon_max, lon_max, lon_min, lon_min]
# rect_lats = [lat_min, lat_min, lat_max, lat_max, lat_min]
# ax[2].plot(rect_lons, rect_lats, color='black', lw=1, transform=ccrs.PlateCarree())

mpl.rcParams['hatch.color'] = 'xkcd:gunmetal'
mpl.rcParams['hatch.linewidth'] = 0.375

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=0.75, 
                    wspace=0.05, 
                    hspace=0.05)

In [ ]:
fig, ax = plt.subplots(4, 3, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ax = [ax[0, 0], ax[0, 1], ax[0, 2], \
      ax[1, 0], ax[1, 1], ax[1, 2], \
      ax[2, 0], ax[2, 1], ax[2, 2], \
      ax[3, 0], ax[3, 1], ax[3, 2]]

letters = ['a', 'b', 'c', \
           'd', 'e', 'f', \
           'g', 'h', 'i', \
           'k', 'l', 'm']

ndsev_nino_bounds = [-3, -2.7, -2.4, -2.1, -1.8, -1.5, -1.2, -0.9, -0.6, -0.3, -0.06, 0.06, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3]
ndsev_nino_norm = c.BoundaryNorm(ndsev_nino_bounds, plt.get_cmap('cmr.fusion_r').N)

cape_nino_bounds = [-100, -90, -80, -70, -60, -50, -40, -30, -20, -10, -2, 2, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
cape_nino_norm = c.BoundaryNorm(cape_nino_bounds, plt.get_cmap('cmr.fusion_r').N)

s06_nino_bounds = [-3, -2.7, -2.4, -2.1, -1.8, -1.5, -1.2, -0.9, -0.6, -0.3, -0.06, 0.06, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3]
s06_nino_norm = c.BoundaryNorm(s06_nino_bounds, plt.get_cmap('cmr.fusion_r').N)

for i in range(12):
    ax[i].coastlines(lw=0.25, color='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0)
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].set_facecolor('xkcd:light gray')
    ax[i].text(-2700000, 1350000, letters[i], fontweight='normal', size=10)

ax[2].pcolormesh(lon, lat, ndsev_pos_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_nino_norm)
ax[8].pcolormesh(lon, lat, ndsev_neg_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_nino_norm)
ax[5].pcolormesh(lon, lat, ndsev_pos_nino.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_nino_norm)
ax[11].pcolormesh(lon, lat, ndsev_neg_nina.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=ndsev_nino_norm)

ax[0].pcolormesh(lon, lat, cape_pos_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_nino_norm)
ax[6].pcolormesh(lon, lat, cape_neg_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_nino_norm)
ax[3].pcolormesh(lon, lat, cape_pos_nino.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_nino_norm)
ax[9].pcolormesh(lon, lat, cape_neg_nina.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=cape_nino_norm)

ax[1].pcolormesh(lon, lat, s06_pos_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_nino_norm)
ax[7].pcolormesh(lon, lat, s06_neg_oni.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_nino_norm)
ax[4].pcolormesh(lon, lat, s06_pos_nino.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_nino_norm)
ax[10].pcolormesh(lon, lat, s06_neg_nina.mean(dim='time')*land, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=s06_nino_norm)

ax[0].text(-3700000, -100000, f"(+) ONI\nn={len(cape_pos_oni)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[6].text(-3700000, -100000, f"(-) ONI\nn={len(cape_neg_oni)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[3].text(-3700000, -100000, f"El Ni\u00f1o\nn={len(cape_pos_nino)}", fontweight='normal', size=10, rotation='horizontal', ha='center')
ax[9].text(-3700000, -100000, f"La Ni\u00f1a\nn={len(cape_neg_nina)}", fontweight='normal', size=10, rotation='horizontal', ha='center')

# ax[0].contourf(lon, lat, cape_pos_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[3].contourf(lon, lat, cape_pos_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[6].contourf(lon, lat, cape_neg_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[9].contourf(lon, lat, cape_neg_nina_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])

# ax[1].contourf(lon, lat, s06_pos_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[4].contourf(lon, lat, s06_pos_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[7].contourf(lon, lat, s06_neg_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[10].contourf(lon, lat, s06_neg_nina_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])

# ax[2].contourf(lon, lat, ndsev_pos_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[5].contourf(lon, lat, ndsev_pos_nino_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[8].contourf(lon, lat, ndsev_neg_oni_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])
# ax[11].contourf(lon, lat, ndsev_neg_nina_sig*land, transform=ccrs.PlateCarree(), colors=['none', 'none'], levels=[0, 0.5, 1], hatches=[None, '////////////'])


cax = plt.axes([0.02, .23, 0.29, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=cape_nino_norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', \
                   ticks=[-100, -50, 0, 50, 100])
cbar.set_label(r'days MAM$^{-1}$', size=10, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.25)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

cax2 = plt.axes([0.355, .23, 0.29, 0.02])
cbar2 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=s06_nino_norm), cax=cax2, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-3, -1.5, 0, 1.5, 3])
cbar2.set_label(r'J kg$^{-1}$', size=10, fontweight='normal', color='black')
cbar2.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar2.outline.set_linewidth(0.25)
cbar2.outline.set_color('black')
# cbar.outline.set_visible(False)

cax3 = plt.axes([0.695, .23, 0.29, 0.02])
cbar3 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=ndsev_nino_norm), cax=cax3, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-3, -1.5, 0, 1.5, 3])
cbar3.set_label(r'm s$^{-1}$', size=10, fontweight='normal', color='black')
cbar3.ax.tick_params(which='both', labelsize=10, width=0.25, length=0, labelcolor='black')
cbar3.outline.set_linewidth(0.25)
cbar3.outline.set_color('black')
# cbar.outline.set_visible(False)

ax[2].set_title('NDSEV', pad=3, fontsize=10)
ax[0].set_title('CAPE', pad=3, fontsize=10)
ax[1].set_title('S06', pad=3, fontsize=10)

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=1.3, 
                    wspace=0.05, 
                    hspace=0.05)